In [42]:
# !git clone https://github.com/stanfordnlp/stanza.git

In [7]:
!pip install stanza

     |████████████████████████████████| 432 kB 8.1 MB/s 
     |████████████████████████████████| 174 kB 52.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=f4187720dee26b47024f310bc7ecc41b0ebc2bcc6f044cebdd11cec19196288c
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [13]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline

# tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
# model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

# nlp = pipeline("ner", model=model, tokenizer=tokenizer)
# example = "Меня зовут Лика и я ем в Италии"

# ner_results = nlp(example)
# print(ner_results)

[{'entity': 'B-LOC', 'score': 0.98588514, 'index': 105, 'word': 'России', 'start': 338, 'end': 344}, {'entity': 'B-LOC', 'score': 0.96787214, 'index': 107, 'word': 'У', 'start': 347, 'end': 348}, {'entity': 'I-LOC', 'score': 0.9497287, 'index': 108, 'word': '##к', 'start': 348, 'end': 349}, {'entity': 'I-LOC', 'score': 0.9709943, 'index': 109, 'word': '##раи', 'start': 349, 'end': 352}, {'entity': 'I-LOC', 'score': 0.9597481, 'index': 110, 'word': '##ну', 'start': 352, 'end': 354}, {'entity': 'B-LOC', 'score': 0.9860973, 'index': 114, 'word': 'Украины', 'start': 374, 'end': 381}, {'entity': 'I-PER', 'score': 0.8532285, 'index': 115, 'word': 'Владимир', 'start': 382, 'end': 390}, {'entity': 'I-PER', 'score': 0.9980101, 'index': 116, 'word': 'З', 'start': 391, 'end': 392}, {'entity': 'I-PER', 'score': 0.9977221, 'index': 117, 'word': '##елен', 'start': 392, 'end': 396}, {'entity': 'I-PER', 'score': 0.9973808, 'index': 118, 'word': '##ский', 'start': 396, 'end': 400}, {'entity': 'B-LOC', 

In [7]:
article_text = 'РОСТОВ-НА-ДОНУ, ул. Митрофанова д.22к1, 2 фев – РИА Новости. Тело мужчины, сбежавшего более двух месяцев назад из Сарпинского психоневрологического диспансера в Калмыкии, обнаружили в поселке неподалеку, сообщает СУСК России по Калмыкии. По информации следствия, вечером 29 ноября мужчина самовольно покинул психдиспансер. По факту безвестного исчезновения мужчины было возбуждено уголовное дело по статье "Убийство". Был организован его розыск."В дополнение к ранее опубликованной информации о розыске подопечного БУРК "Сарпинский психоневрологический диспансер" Строкова А.Н. региональное следственное управление СК России сообщает, что на территории поселка Годжур Сарпинского района обнаружено тело мужчины", - говорится в сообщении.В настоящее время по данному факту организована проверка.'

In [46]:
import stanza
from graph import InferenceWorker
import pymorphy2
import json
stanza.download('ru')
morph = pymorphy2.MorphAnalyzer()

In [47]:
class SampleNER(InferenceWorker):
    def __call__(self, t, d):
        if t == "text":
            yield "entities", self.ner_from_text(d), "NER"
        else:
            yield None        
    def ner_from_text(self, text):
        nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
        doc = nlp(article_text)
        ner_ents= []
        for sent in doc.sentences:
            for ent in sent.ents:  
                ent = json.loads(str(ent))
                ner_ents.append([ent['text'], ent['type']])
        for x in ner_ents:
          if x[1] == 'LOC':
            if x[0].isalpha():
              x[0] = morph.normal_forms(x[0])[0].capitalize()
            else:
              x[0] = x[0].capitalize()
        return ner_ents
        

In [48]:
worker = SampleNER()
worker.serialize()

Worker saved!


In [49]:
for x in worker("text", article_text):
    print(x)

2022-02-12 23:08:38 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2022-02-12 23:08:38 INFO: Use device: cpu
2022-02-12 23:08:38 INFO: Loading: tokenize
2022-02-12 23:08:38 INFO: Loading: ner
2022-02-12 23:08:40 INFO: Done loading processors!


('entities', [['Ростов-на-дону', 'LOC'], ['Ул. митрофанова', 'LOC'], ['РИА Новости', 'ORG'], ['Сарпинский', 'LOC'], ['Калмыкия', 'LOC'], ['СУСК России', 'ORG'], ['Калмыкия', 'LOC'], ['Убийство', 'MISC'], ['БУРК "Сарпинский психоневрологический диспансер', 'ORG'], ['Строкова А.Н.', 'PER'], ['СК России', 'ORG'], ['Годжур', 'LOC'], ['Сарпинского района', 'LOC']], 'NER')
